## 2 - Updated Sentiment Analysis

Ở Notebook trước, chúng ta đã học về các kiến thức nền tảng của phân tích cảm xúc. Trong notebook lần này, chúng ta sẽ phân tích và cố gắng đạt được kết quả tốt hơn. <br>
Dưới đây là danh sách công việc sẽ thực hiện trong notebook này: <br>
* Gói, nhóm các chuỗi đã được đệm vào với nhau.
* Sử dụng pre-trained word embedding.
* Sử dụng cấu trúc RNN khác.
* Sử dụng cấu trúc bidirectional RNN.
* Sử dụng multi-layer RNN.
* Áp dụng Regularization.
* Sử dụng Optimizer khác. <br>

***=> Sau khi thực hiện các phương pháp trên, chúng ta sẽ đạt được accuracy khoảng 84%***

### Preparing Data

> Tương tự như ở notebook 1, chúng ta sẽ định nghĩa ***Fields*** và chia train/valid/test. <br>
Chúng ta sẽ sử dụng *packed padded sequences* - thứ khiến mô hình RNN chỉ xử lý các phần tử *non-padded* trong chuỗi, bất cứ phần tử nào đã được padded, đầu ra sẽ là một zero tensor. Để sử dụng *packed padded sequences*, chúng ta cần cho RNN biết trước chiều dài của chuỗi thực. Để làm được việc đó, chúng ta cho ***include_lengths=True*** trong trường ***TEXT***. Sau khi làm như vậy, ***batch.text*** bây giờ là một tupple với phần tử đầu tiên là sentences của chúng ta (các tensor dạng số đã được đệm) và phần tử thứ 2 là chiều dài thực tế của sentences.

In [1]:
import torch
from torchtext.legacy import data

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy',
                    tokenizer_language='en_core_web_sm',
                    include_lengths = True)

LABEL = data.LabelField(dtype = torch.float)

> Tải dữ liệu IDMb

In [2]:
from torchtext.legacy import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

> Tạo validation từ tập train

In [3]:
import random
train_data, valid_data = train_data.split(random_state = random.seed(SEED))

> Tiếp theo, chúng ta sẽ sử dụng pre-trained word embeddings. Thay vì khởi tạo word embeddings một cách ngẫu nhiên, ta khởi tạo chúng bằng các vectors đã được huấn luyện trước. Chỉ cần set đối số ***build_vocab=tên vector***, Torchtext sẽ tự động tải và liên kết chúng với các từ trong từ điển của chúng ta. <br>
> Ở đây, chúng ta sẽ sử dụng ***glove.6B.100d*** vectors. ***glove*** là một thuật toán dùng để tính toán các vectors, chi tiết xem thêm tại [đây](https://nlp.stanford.edu/projects/glove/). 6B ám chỉ có 6 tỉ tokens, 100d nghĩa là vector có 100 chiều. <br>
> Theo lý thuyết, các pre-trained vectors mang hàm ý: các từ có nghĩa gần nhau sẽ nằm gần nhau trong không gian vector. Việc sử dụng pre-trained vector giúp cho embedding layer có giá trị khởi tạo tốt và không phải học mối quan hệ giữa các từ từ đầu. <br>
> Lưu ý, Torchtext sẽ mặc định khởi tạo các từ có trong từ điển NHƯNG KHÔNG CÓ TRONG PRE-TRAINED EMBEDDING giá trị 0. Tuy nhiên chúng ta sẽ khởi tạo ngẫu nhiên chúng theo phân phối chuẩn Gaussian bằng cách sử dụng ***unk_init = torch.Tensor.normal_***

In [4]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data,
                 max_size = MAX_VOCAB_SIZE,
                 vectors = "glove.6B.100d",         # sử dụng pre-trained embedding
                 unk_init = torch.Tensor.normal_)   # khởi tạo các từ không có trong pre-trained embedding theo phân phối chuẩn

LABEL.build_vocab(train_data)
                

> Tiếp theo, chúng ta tạo iterator và sắp xếp các *packed padded sequences* theo độ dài của senteces bằng cách ***sort_within_batch = True***.

In [5]:
BATCH_SIZE = 64
from torchtext.legacy import data

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator,\
valid_iterator,\
test_iterator = \
    data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    device = device,
    sort_within_batch = True)


### Build the Model

> Lần này, các feature của model sẽ thay đổi đáng kể.

##### Different RNN Architecture

>Lần này, chúng ta sẽ sử dụng một kiến trúc RNN khác gọi là Long Short - Term Memory (LSTM). Kiến trúc RNNs tiêu chuẩn (như ở notebook 1) gặp phải một vấn đề khá nghiêm trọng là [vanishing gradient problem](https://en.wikipedia.org/wiki/Vanishing_gradient_problem). Tuy nhiên, LSTM xử lý vấn đề này bằng cách tạo thêm một recurrent state được gọi là cell (c). Có thể hiểu cell như một memory trong LSTM - sử dụng nhiều gates để điều khiển luồng thông tin vào/ra memory. Nhấn vào [đây](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) để biết thêm chi tiết. Hiểu một cách đơn giản, LSTM là một hàm của $x_t, h_t, c_t$ thay vì $x_t, h_t$ như RNNs. <br>
* ($h_t, c_t$) = LSTM($x_t, h_t, c_t$) <br>
> Cấu trúc của LSTM như hình sau:

![LSTM](./images/2_lstm_figure1.PNG "LSTM")

> Cell state được khởi tạo bằng tensor 0. Tuy nhiên, quá trình prediction chỉ sử dụng final hidden state, không sử dụng final cell state: $\hat{y}$ = f($h_T$)

##### Bidirectional RNN

> Khái niệm bidirectional RNN khá là đơn giản. Thay vì chỉ áp dụng RNN từ đầu tới cuối sentence (gọi là **forward RNN**) ta thực hiện thêm một bước là áp dụng RNN từ cuối cho tới đầu câu (gọi là **backward RNN**). Tại thời điểm t, **forward RNN** xử lý từ $x_t$, trong khi **backward RNN** xử lý từ $x_{T - t + 1}$. <br>
> Trong pytorch, hidden state (và cell state) tensors được trả về bởi forward RNN và backward RNN được xếp chồng lên nhau. <br>
> Ta dự đoán dựa trên last hidden state của forward RNN: $h_{T forward}$ và last hidden state của backward RNN: $h_{T backward}$. 
* $\hat{y}$ = f($h_{T forward}$, $h_{T backward}$)

> Hình dưới mô tả cấu trúc bidirectional RNN với: màu xanh biểu thị backward, màu vàng biểu thị forward và linear layer được biểu thị bởi màu bạc.

![LSTM](./images/2_bi-rnn_figure2.PNG "LSTM")

##### Multi-layer RNN

> Multi-layer RNNs còn được gọi là deep RNNs. Ý tưởng ở đây là ta thêm các lớp RNNs bên trên lớp RNN tiêu chuẩn. Hidden state của lớp RNN phía dưới, tại thời điểm t sẽ là đầu vào của lớp RNN phía trên, cũng tại thời điểm t. Quá trình prediction được thực hiện ở *last hidden state* của *last layer* (layer cao nhất). <br>
> Hình dưới minh họa kiến trúc multi-layer undirectional RNN, số layer là superscript (chỉ số trên). Lưu ý rằng mỗi layer cũng cần có giá trị khởi tạo hidden state ban đầu của riêng nó: ${h_0}^0$ và ${h_0}^1$

![LSTM](./images/2_multilayer_rnn_figure3.PNG "LSTM")

##### Regularization

> Khi chúng ta thêm nhiều layer cũng như sử dụng bidirectional, mô hình của chúng ta có thêm nhiều parameters. Và, "the more parameters you have in your model, the higher the probability that your model will overfit". Để giải quyết vấn đề này, chúng ta sử dụng regularization, cụ thể là *dropout*. <br>
> *Dropout* hoạt động bằng cách lựa chọn ngẫu nhiên một số neural trong layer và cho chúng bằng 0 trong quá trình lan truyền thuận. Số lượng neural bị drop là 1 hyperparameter. <br>
> Có một lý thuyết chỉ ra rằng lý do dropout có thể xử lý vấn đề overfitting: Khi drop 1 số lượng neural nhất định, model sau khi bị dropout sẽ có số lượng parameters ít hơn (gọi là weaker model). Prediction của weaker model được lấy trung bình bên trong model. Do đó model gốc có thể được coi là sự kết hợp của các weaker models, và các weaker models này không bị overfitting.

##### Implementation Details

> Chúng ta bổ sung thêm một nội dung khác cho mô hình này là: mô hình sẽ không học embedding đối với tokens. Lý do là việc padding tokens không liên quan gì tới việc xác định cảm xúc, quan điểm trong câu. Điều này có nghĩa *pad token* vẫn giữ nguyên giá trị khởi tạo của chúng. Ta thực hiện việc này bằng cách truyền thêm tham số ***padding_idx*** vào layer ***nn.Embedding***.

> Chúng ta sẽ sử dụng LSTM thay cho RNN (`nn.LSTM` thay cho `nn.RNN`). Lưu ý là LSTM trả về một tupple chứa: ***output, final hidden state, final cell state*** trong khi RNN chỉ trả về ***output, final hidden state***.

> Do final hidden state trong LSTM có cả forward và backward, đầu ra được ghép nối với nhau nên kích thước đầu vào của ***nn.Linear*** gấp 2 lần kích thước của hidden.

> Việc triển khai bidirectionality và thêm các layer khác được thực hiện bằng cách thiết lập 2 tham số: `num_layers` và `bidirectional` trong RNN/LSTM.

> Triển khai dropout bằng ***nn.Dropout*** layer, áp dụng dropout sau mỗi layer mà ta muốn thực hiện dropout. <br>
> **NOTE**: ***KHÔNG BAO GIỜ ÁP DỤNG DROPOUT CHO ĐẦU VÀO VÀ ĐẦU RA, CHỈ ÁP DỤNG DROPOUT CHO CÁC LỚP TRUNG GIAN***. <br>
> LSTM có tham số `dropout` cho phép thêm dropout vào lớp kết nối giữa các hidden state trong một layer và hidden state trong layer kế tiếp.

> Do áp dụng *packed padded sequences*, ta thêm đối số thứ 2 vào hàm ***foward*** là ***text_lengths***.

> Trước khi truyền embeddings vào RNN, chúng ta cần gói chúng lại bằng cách sử dụng: ***nn.utils.rnn.packed_padded_sequence***. Việc này khiến RNN chỉ xử lý các non-padded elements trong sequence. RNN sẽ trả về ***packed_output***, ***hidden*** và ***cell*** states. Nếu không sử dụng packed padded sequences, ***hidden*** và ***cell*** là các tensor từ last element trong sequence - khả năng cao là pad token. Tuy nhiên khi sử dụng packed padded sequences, ***hidden*** và ***cell*** là tensors từ last non-padded element trong sequence. Lưu ý: ***lengths*** và ***packed_padded_sequence*** phải được chuyển về CPU tensor bằng cách `.to('cpu')`.

> Sau đó, chúng ta cần unpack chuỗi đầu ra bằng ***nn.utils.rnn.pad_paced_sequence*** để biến đổi chúng từ dạng packed sequence sang tensor. Các phần tử trong ***output*** từ các padding tokens sẽ có giá trị tensor 0.

> final hidden state, ***hidden*** có kích thước: [**num layers * num directions, batch_size, hidden dim**]. Thứ tự của chúng sẽ là:  [**forward_layer_0, backward_layer_0, forward_layer_1, backward_layer 1, ..., forward_layer_n, backward_layer n**]. <br>
> Chúng ta cần lấy final layer foward và final layer backward hidden states, ta sẽ lấy 2 lớp cuối cùng bằng 2 câu lệnh sau: ***hidden[-2,:,:]*** và ***hidden[-1,:,:]***, sau đó ghép nối chúng và truyền vào linear layer (sau khi áp dụng dropout).

In [6]:
import torch.nn as nn
class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        # Khi truyền vào tham số padding_idx, embedding sẽ không tính toán gradient cho các từ có index bằng padding_idx
        # ở đây là các pad tokens vì pad tokens không ảnh hưởng gì đến quá trình nhận định cảm xúc trong câu.
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.rnn = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout)
        
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        
        # text = [sent len, batch size]
        
        embedded = self.dropout(self.embedding(text))
        
        # embedded = [sent len, batch size, emb dim]
        
        # pack sequence
        # lengths need to be on CPU!
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.to('cpu'))
        
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        
        # unpack sequence
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)

        # output = [sent len, batch size, hid dim * num directions]
        # output over padding tokens are zero tensors
        
        # hidden = [num layers * num directions, batch size, hid dim]
        # cell = [num layers * num directions, batch size, hid dim]
        
        # concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        # and apply dropout
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
                
        #hidden = [batch size, hid dim * num directions]
            
        return self.fc(hidden)

> Để đảm bảo pre-trained vetors được load vào model, ***EMBEDDING_DIM*** phải bằng với số chiều của pre-trained vector. <br>
> Ta lấy chỉ số của pad tokens trong từ điển bằng attribute ***pad_token***.

In [7]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX)

In [8]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 4,810,857 trainable parameters


In [9]:
# We retrieve the embeddings from the field's vocab, and check they're the correct size, [vocab size, embedding dim]
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([25002, 100])


> Sau đó, chúng ta sẽ thay thế weights của embedding layer bằng pre-trained embeddings <br>
> NOTE: phải dùng ***weight.data***, KHÔNG phải ***weight***!

In [10]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.1386,  0.1180,  0.3534,  ...,  0.1226,  0.5973, -0.1702],
        [-0.0786,  0.0541, -0.0993,  ...,  0.2565, -0.1874, -0.4428],
        [-0.3617,  0.6201,  0.1105,  ...,  0.2994, -0.5920,  1.0949]])

> Khởi tạo các token không có trong từ điển pre-trained và pad token có giá trị là tensor 0 do chúng không liên quan gì đến việc phân biệt cảm xúc:

In [11]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.1386,  0.1180,  0.3534,  ...,  0.1226,  0.5973, -0.1702],
        [-0.0786,  0.0541, -0.0993,  ...,  0.2565, -0.1874, -0.4428],
        [-0.3617,  0.6201,  0.1105,  ...,  0.2994, -0.5920,  1.0949]])


### Train the Model

> Thay vì sủ dụng SGD, notebook này sẽ sử dụng Adam. nhấn vào [đây](http://ruder.io/optimizing-gradient-descent/index.html) để hiểu thêm về thuật toán Adam.

In [12]:
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

> Tiếp theo, ta cần định nghĩa hàm loss (trong pytorch thường gọi là criterion). <br>
Do đầu ra là 0 hoặc 1, ta sử dụng Binary cross entropy.

In [13]:
criterion = nn.BCEWithLogitsLoss()

In [14]:
model = model.to(device)
criterion = criterion.to(device)

In [15]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

> * Hàm ***train*** lặp xuyên suốt training set, mỗi một batch một lần. <br>
***model.train()*** đưa mô hình vào chế độ huấn luyện - bật _drop out_ và _batch normalization_. <br>
> * Với mỗi batch training, đầu tiên ta cần đưa gradients về 0. Mỗi tham số của mô hình có thuộc tính ***grad*** lưu trữ gradient được tính bởi ***creterion***. Pytorch không tự động đưa gradient về 0, do đó ta cần thực hiện. <br>
> * Lưu ý, khi đưa một batch các câu, ***batch.text*** vào mô hình, ta không cần dùng phương thức ***forward***. ***squeeze*** cần thiết bởi việc dự đoán được khởi tạo với kích thước ***[batch size, 1]***, và ta cần xóa bỏ chiều 1, predictions yêu cầu đầu vào có kích thước ***[batch size]***. <br>
> * Loss và accuracy được tính toán bằng cách sử dụng đầu ra của mô hình và label của batch - ***batch.label***, loss được tính trung bình trên toàn batch. <br>
> * Ta tính gradient của mỗi tham số bằng ***loss.backward()***, sau đó cập nhật gradient bằng ***optimizer.step()***.

> * Khi set ***include_lengths=True***, ***batch.text*** là một tupple với phần tử đầu tiên là một tensor, phần tử thứ 2 là chiều dài thực của sequence. Ta chia chúng vào 2 biến text và text_lengths trước khi truyền chúng vào model. <br>
> * **NOTE**: Nhớ dùng model.train() để bật dropout trong lúc train.

In [16]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        text, text_lengths = batch.text
        
        predictions = model(text, text_lengths).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [17]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text, text_lengths = batch.text
            
            predictions = model(text, text_lengths).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [18]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

> Fnially, we train model =))

In [19]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), './models/tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

In [ ]:
model.load_state_dict(torch.load('./models/tut2-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

### User Input

> Hàm ***predict_sentiment*** thực hiện các công việc sau:
* Đưa model vào chế độ evaluation.
* tokenizes các sentence.
* Đánh chỉ số tokens bằng cách chuyển chúng thành các số biểu diễn chúng trong vocab.
* Lấy length của sequence.
* Chuyển chỉ số từ Python list sang Pytorch tensor.
* Thêm 1 chiều batch với `unsqueeze`.
* Chuyển length về dạng tensor.
* Chuyển đầu ra về dạng nhị phân với `sigmoid`.
* Chuyển tensor chứa 1 giá trị đơn lẻ về dạng integer với phương thức `item()`.

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor, length_tensor))
    return prediction.item()

In [ ]:
predict_sentiment(model, "This film is terrible")

In [ ]:
predict_sentiment(model, "This film is great")

### Next Steps

> Trong notebook tiếp theo, chúng ta sẽ xây dựng một mô hình có độ chính xác tốt hơn và thời gian huấn luyện nhanh hơn.